IDRID with Dropout-Black and White

In [ ]:
import pandas as pd
import numpy as np
import cv2
from skimage import exposure
from scipy.ndimage import gaussian_filter
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import models, layers
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import LearningRateScheduler
import math
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
import pandas as pd
import cv2
from skimage import exposure
from scipy.ndimage import gaussian_filter
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

In [ ]:
# Load the dataset
data = pd.read_csv("test_split_image_key.csv")

# Split the dataset into training and testing sets
train_data = data[data["split_type"] == "training"]
test_data = data[data["split_type"] == "testing"]

# Define output directories for preprocessed images
output_dir_train = "preprocessed_images_bw/train/"
output_dir_test = "preprocessed_images_bw/test/"

# Create directories if they don't exist
os.makedirs(output_dir_train, exist_ok=True)
os.makedirs(output_dir_test, exist_ok=True)

In [ ]:
import os
import cv2
from scipy.ndimage import gaussian_filter


# Preprocessing function
def preprocess_image(image_path, output_dir):
    # Load and resize the image to 512x512
    image = cv2.imread(image_path)
    resized_image = cv2.resize(image, (512, 512))

    # Convert image to grayscale (black and white)
    gray_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)

    # Optionally, you can still apply Gaussian filter to the grayscale image to remove noise
    filtered_image = gaussian_filter(gray_image, sigma=1)

    # Save the preprocessed grayscale image
    image_name = os.path.basename(image_path)
    output_path = os.path.join(output_dir, image_name)
    cv2.imwrite(output_path, filtered_image)


# Apply preprocessing to training and testing images

for img_name in train_data["Image name"]:
    image_path = "a_training/" + img_name + ".jpg"
    preprocess_image(image_path, output_dir_train)

for img_name in test_data["Image name"]:
    image_path = "b_testing/" + img_name + ".jpg"
    preprocess_image(image_path, output_dir_test)

In [ ]:
# Convert the images to numpy arrays
X_train = []
X_test = []

for img_name in train_data["Image name"]:
    image_path = os.path.join(output_dir_train, img_name + ".jpg")
    image = cv2.imread(image_path)
    X_train.append(image)

for img_name in test_data["Image name"]:
    image_path = os.path.join(output_dir_test, img_name + ".jpg")
    image = cv2.imread(image_path)
    X_test.append(image)

X_train = np.array(X_train)
X_test = np.array(X_test)

# Extract labels directly without one-hot encoding
y_train = train_data["Retinopathy grade"].values
y_test = test_data["Retinopathy grade"].values

In [ ]:
input_shape = (512, 512, 3)
num_classes = 5
epochs = 10  # placeholder

In [ ]:
cnn512_dropout = models.Sequential(
    [
        keras.Input(shape=input_shape),
        # Input Layer	Zero Padding layer	Padding (2,2)
        layers.ZeroPadding2D(padding=(2, 2)),
        # Layer 1	2D CONV layer	Kernel number = 32, kernel size = 3
        layers.Conv2D(32, kernel_size=(3, 3)),
        # Layer 2	Batch Normalization layer	-
        layers.BatchNormalization(),
        # Layer 3	Relu layer	-
        layers.Activation("relu"),
        # Layer 4	Max Pooling layer	Pooling size (2,2)
        layers.MaxPooling2D(pool_size=(2, 2)),
        # Layer 5	2D CONV layer	Kernel number = 64, kernel size = 3
        layers.Conv2D(64, kernel_size=(3, 3)),
        # Layer 6	Batch Normalization layer	-
        layers.BatchNormalization(),
        # Layer 7	Relu layer	-
        layers.Activation("relu"),
        # Layer 8	Max Pooling layer	Pooling size (2, 2)
        layers.MaxPooling2D(pool_size=(2, 2)),
        # Layer 9	2D CONV layer	Kernel number = 96, kernel size = 3
        layers.Conv2D(96, kernel_size=(3, 3)),
        # Layer 10	Batch Normalization layer	-
        layers.BatchNormalization(),
        # Layer 11	Relu layer	-
        layers.Activation("relu"),
        # Layer 12	Max Pooling layer	Pooling size (2, 2)
        layers.MaxPooling2D(pool_size=(2, 2)),
        # Layer 13	2D CONV layer	Kernel number = 96, kernel size = 3
        layers.Conv2D(96, kernel_size=(3, 3)),
        # Layer 14	Batch Normalization layer	-
        layers.BatchNormalization(),
        # Layer 15	Relu layer	-
        layers.Activation("relu"),
        # Layer 16	Max Pooling layer	Pooling size (2, 2)
        layers.MaxPooling2D(pool_size=(2, 2)),
        # Layer 17	2D CONV layer	Kernel number = 128, kernel size = 3
        layers.Conv2D(128, kernel_size=(3, 3)),
        # Layer 18	Batch Normalization layer	-
        layers.BatchNormalization(),
        # Layer 19	Relu layer	-
        layers.Activation("relu"),
        # Layer 20	Max Pooling layer	Pooling size (2, 2)
        layers.MaxPooling2D(pool_size=(2, 2)),
        # Layer 21	2D CONV layer	Kernel number = 200, kernel size = 3
        layers.Conv2D(200, kernel_size=(3, 3)),
        # Layer 22	Batch Normalization layer	-
        layers.BatchNormalization(),
        # Layer 23	Relu layer	-
        layers.Activation("relu"),
        # Layer 24	Max Pooling layer	Pooling size (2, 2)
        layers.MaxPooling2D(pool_size=(2, 2)),
        # Layer 25	Flatten layer	-
        layers.Flatten(),
        # Layer 26	FC layer	Neurons number = 1000
        layers.Dense(1000),
        # Layer 27	Batch Normalization layer	-
        layers.BatchNormalization(),
        # Layer 28	Relu layer	-
        layers.Activation("relu"),
        layers.Dropout(0.5),
        # Layer 29	FC layer	Neurons number = 500
        layers.Dense(500),
        # Layer 30	Batch Normalization layer	-
        layers.BatchNormalization(),
        # Layer 31	Relu layer	-
        layers.Activation("relu"),
        layers.Dropout(0.5),
        # Layer 32	FC layer with Softmax activatio
        layers.Dense(num_classes, activation="softmax"),
    ]
)

In [ ]:
from tensorflow.keras.optimizers import SGD

# base learning rate for custom CNNs
base_learning_rate = 1e-4
# maximum learning rate for custom CNNs
max_learning_rate = 1e-1

# Create an instance of SGD optimizer with initial learning rate
optimizer = SGD(learning_rate=base_learning_rate, momentum=0.9)

cnn512_dropout.compile(
    optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)

# create class weight
classes = np.unique(y_train)
class_weights = compute_class_weight("balanced", classes=classes, y=y_train)
class_weight_dict = dict(zip(classes, class_weights))


# create triangular schedule
def triangular_schedule(epoch):
    """Triangular learning rate scheduler."""
    cycle_length = 10  # Define the length of a cycle
    cycle = math.floor(1 + epoch / (2 * cycle_length))
    x = abs(epoch / cycle_length - 2 * cycle + 1)
    lr = base_learning_rate + (max_learning_rate - base_learning_rate) * max(0, (1 - x))
    return lr


# When fitting the model, include the learning rate scheduler callback
lr_scheduler = LearningRateScheduler(triangular_schedule)

In [ ]:
# fitting the model
history_cnn512_dropout = cnn512_dropout.fit(
    X_train,
    y_train,
    epochs=epochs,
    class_weight=class_weight_dict,
    callbacks=[lr_scheduler],
)

In [ ]:
# Evaluate the model
test_loss, test_accuracy = cnn512_dropout.evaluate(X_test, y_test)
print("Test Accuracy:", test_accuracy)